<a href="https://colab.research.google.com/github/slsabilaAura/Tugas-Akhir/blob/main/topik_pake_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install gensim

In [2]:
# prompt: import file form drive

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
import pandas as pd
# Save the preprocessed DataFrames to new CSV files
proposal_df = pd.read_csv('/content/drive/MyDrive/Skripsi3/Dataset/processed_proposalC.csv')
expert_df = pd.read_csv('/content/drive/MyDrive/Skripsi3/Dataset/processed_expertC.csv')

In [4]:
import numpy as np
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
import ast
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# Load Data

In [5]:
def convert_to_list(text):
    try:
        return ast.literal_eval(text) if isinstance(text, str) else text
    except:
        return []


proposal_df["processed"] = proposal_df["stemmed"].apply(convert_to_list)
expert_df["processed"] = expert_df["stemmed"].apply(convert_to_list)

# Gabungkan semua dokumen untuk membuat satu kamus bersama
documents_all = proposal_df["processed"].tolist() + expert_df["processed"].tolist()
dictionary_all = Dictionary(documents_all)

dictionary_all.filter_extremes(no_below=5, no_above=0.5)

proposal_corpus = [dictionary_all.doc2bow(doc) for doc in proposal_df["processed"]]
expert_corpus = [dictionary_all.doc2bow(doc) for doc in expert_df["processed"]]

corpus_all= proposal_corpus + expert_corpus

In [6]:
print(len(documents_all))
print(len(corpus_all))
print(len(dictionary_all))

609
609
3086


# Model LDA

In [ ]:
# import time
# import pandas as pd
# import pandas as pd
# from gensim.models.ldamodel import LdaModel
# from gensim.models.coherencemodel import CoherenceModel

# # List untuk menyimpan hasil eksperimen
# results_lda = []


# for num_topics in range(1, 51):
#     start_time = time.time()  # Mulai timer

#     # Train LDA model
#     lda_model_lda = LdaModel(
#         corpus=corpus_all,
#         id2word=dictionary_all,
#         num_topics=num_topics,
#         alpha=0.5,   # Bisa diganti dengan nilai tetap
#         eta=0.01,     # Bisa diganti dengan nilai tetap
#         passes=50,
#         random_state=42,
#         iterations= 400,
#     )

#     # Evaluasi dengan coherence score
#     coherence_model_lda= CoherenceModel(
#         model=lda_model_lda,
#         texts=proposal_df['processed'].tolist() + expert_df['processed'].tolist(),
#         dictionary=dictionary_all,
#         coherence='c_v'
#     )
#     coherence_score_lda = coherence_model_lda.get_coherence()

#     # Evaluasi Topic Diversity
#     top_words_per_topic_lda = [lda_model_lda.show_topic(topic_id, topn=20) for topic_id in range(num_topics)]
#     unique_words_lda = set([word for topic in top_words_per_topic_lda for word, _ in topic])
#     total_words_lda = num_topics * 20  # 10 kata per topik
#     topic_diversity_lda = len(unique_words_lda) / total_words_lda if total_words_lda > 0 else 0

#     # Hitung waktu eksekusi
#     time_taken_lda = time.time() - start_time

#     # Simpan hasil dalam list
#     results_lda.append([num_topics, coherence_score_lda, topic_diversity_lda, time_taken_lda])
#     print(f"Topics: {num_topics} | Coherence: {coherence_score_lda:.4f} | Diversity: {topic_diversity_lda:.4f} | Time: {time_taken_lda:.2f} sec")

# # Simpan ke DataFrame
# results_looping_df = pd.DataFrame(results_lda, columns=['num_topics', 'coherence_score_lda', 'topic_diversity_lda', 'time_taken_lda'])

In [ ]:
# results_looping_df.to_csv('/content/drive/MyDrive/Skripsi4/topik/dictionary/looping_baru.csv', index=False, encoding='utf-8-sig')

In [7]:
from itertools import combinations

num_topics = 30  # atau optimalisasi via coherence

lda_model = LdaModel(
    corpus=corpus_all,
    id2word=dictionary_all,
    num_topics=num_topics,
    passes=50,
    random_state=42,
    iterations= 400,
    alpha=0.5,
    eta=0.01

)


In [8]:
from gensim.matutils import sparse2full

# Fungsi untuk mendapatkan dense topic vector
def get_topic_vector(lda_model, dictionary, document, num_topics):
    bow = dictionary.doc2bow(document)
    topic_dist = lda_model.get_document_topics(bow, minimum_probability=0.00001)
    return sparse2full(topic_dist, num_topics)

# Ambil jumlah topik dari model LDA
num_topics = lda_model.num_topics

# Hitung topik vektor dalam bentuk dense array (untuk expert)
expert_df["topic_vector"] = [
    get_topic_vector(lda_model, dictionary_all, doc, num_topics)
    for doc in expert_df["processed"]
]

# Hitung topik vektor dalam bentuk dense array (untuk proposal)
proposal_df["topic_vector"] = [
    get_topic_vector(lda_model, dictionary_all, doc, num_topics)
    for doc in proposal_df["processed"]
]


In [10]:
proposal_df["tahun"] = proposal_df["proposal_year"].astype(int)
expert_df["pub_year"] = expert_df["research_pub_year"].astype(int)


def similarity_m_to_d(proposal_vector, expert_vector):
    numerator = np.dot(proposal_vector, expert_vector)
    denominator = (np.linalg.norm(proposal_vector) + 1e-10)
    return numerator / (denominator)


def similarity_d_to_m(expert_vector, proposal_vector):
    numerator = np.dot(expert_vector, proposal_vector)
    denominator = (np.linalg.norm(expert_vector) + 1e-10)
    return numerator / (denominator)

def time_decay(year_prop, year_ex, t=1, gamma=0.1):
    decay = 1 - ((year_prop - year_ex) / t) * gamma
    return max(decay, 0.0)  # tidak boleh negatif

# mapping Dosen dengan ID Dosen

dosen_id_map = pd.read_csv("/content/drive/MyDrive/Skripsi3/Dataset/mapping.csv")  # pastikan kolom: expert_id, expert_name
dosen_id_map["expert_name"] = dosen_id_map["expert_name"].str.strip().str.lower()


def explode_authors_with_weights(df, dosen_id_map):
    rows = []

    # Normalisasi nama dosen agar cocok
    dosen_id_map = dosen_id_map.copy()
    dosen_id_map["expert_name"] = dosen_id_map["expert_name"].str.strip().str.lower()

    for _, row in df.iterrows():
        authors = row.get("authors", [])
        # Bersihkan nama kosong atau NaN
        authors = [a for a in authors if isinstance(a, str) and a.strip() != ""]

        num_authors = len(authors)
        for idx, name in enumerate(authors):
            name_clean = name.strip().lower()

            if num_authors == 1:
                weight = 1.0
            else:
                weight = 0.6 if idx == 0 else 0.4 / (num_authors - 1)

            new_row = row.to_dict()
            new_row["name"] = name
            new_row["author_position"] = idx + 1
            new_row["num_authors"] = num_authors
            new_row["author_weight"] = round(weight, 4)

            matched = dosen_id_map[dosen_id_map["expert_name"] == name_clean]
            new_row["expert_id"] = matched["expert_id"].values[0] if not matched.empty else None

            rows.append(new_row)

    return pd.DataFrame(rows)


# Gabungkan author_1 sampai author_6 jadi list
author_cols = ["author_1", "author_2", "author_3", "author_4", "author_5", "author_6"]
expert_df["authors"] = expert_df[author_cols].values.tolist()

# Hapus duplikat berdasarkan research_id
expert_df = expert_df.drop_duplicates(subset=["research_id"]).copy()

# Jalankan explode
expert_df = explode_authors_with_weights(expert_df, dosen_id_map)

# Opsional: hanya simpan baris dengan expert_id valid
expert_df = expert_df[expert_df["expert_id"].notna()]




In [11]:
import numpy as np
import pandas as pd

topic_matrix_expert = np.vstack(expert_df["topic_vector"])
topic_df = pd.DataFrame(
    topic_matrix_expert,
    columns=[f"topic_{i+1}" for i in range(lda_model.num_topics)]
)

topic_df["research_id"] = expert_df["research_id"].values

topic_df.set_index("research_id", inplace=True)
print(topic_df)
# topic_df.to_csv('/content/drive/MyDrive/Skripsi4/dictionary/vektorExpert_30.csv', index=False)


              topic_1   topic_2   topic_3   topic_4   topic_5   topic_6  \
research_id                                                               
R1           0.036934  0.087059  0.162502  0.009125  0.007667  0.005118   
R1           0.036934  0.087059  0.162502  0.009125  0.007667  0.005118   
R2           0.006545  0.471147  0.005548  0.145334  0.008565  0.005310   
R3           0.006596  0.006009  0.005173  0.011906  0.006250  0.005892   
R3           0.006596  0.006009  0.005173  0.011906  0.006250  0.005892   
...               ...       ...       ...       ...       ...       ...   
R464         0.007638  0.008468  0.007235  0.007249  0.008771  0.006685   
R464         0.007638  0.008468  0.007235  0.007249  0.008771  0.006685   
R465         0.007437  0.019320  0.113598  0.007250  0.007641  0.007323   
R466         0.007806  0.024004  0.060841  0.006231  0.007779  0.044248   
R467         0.035698  0.004191  0.004376  0.031360  0.004495  0.005574   

              topic_7   

In [12]:
topic_matrix_proposal = np.vstack(proposal_df["topic_vector"])
topic_proposal_df = pd.DataFrame(topic_matrix_proposal, columns=[f"topic_{i+1}" for i in range(lda_model.num_topics)])
topic_proposal_df["proposal_id"] = proposal_df["proposal_id"].values

# Set kolom id_dosen sebagai index
topic_proposal_df.set_index("proposal_id", inplace=True)
print(topic_proposal_df)
# topic_proposal_df.to_csv('/content/drive/MyDrive/Skripsi4/dictionary/vektorProposal_30.csv', index=False)

              topic_1   topic_2   topic_3   topic_4   topic_5   topic_6  \
proposal_id                                                               
P1           0.000480  0.000403  0.000377  0.000371  0.000397  0.000356   
P10          0.000311  0.000315  0.000309  0.000329  0.000339  0.000282   
P11          0.000296  0.000328  0.129495  0.000294  0.000330  0.002315   
P12          0.000298  0.000340  0.061452  0.000436  0.000404  0.041874   
P13          0.000413  0.000604  0.000466  0.000439  0.000457  0.000439   
...               ...       ...       ...       ...       ...       ...   
P138         0.000549  0.000407  0.000402  0.000414  0.000430  0.000382   
P139         0.004678  0.000600  0.000755  0.000581  0.006652  0.013836   
P140         0.000468  0.000449  0.000388  0.011474  0.000424  0.000355   
P141         0.066119  0.001308  0.001186  0.001534  0.001554  0.001019   
P142         0.026073  0.001327  0.050764  0.006353  0.000914  0.000736   

              topic_7   

# Kemiripan

In [13]:
import pandas as pd
import ast

# Load data
df = pd.read_csv("/content/drive/MyDrive/Skripsi4/dictionary/exploded.csv")

import re
import ast

def fix_vector(text):
    if isinstance(text, str):
        # Tambahkan koma antara angka yang dipisah spasi
        text_fixed = re.sub(r'(?<=\d)\s+(?=\d)', ', ', text.strip())
        try:
            return ast.literal_eval(text_fixed)
        except:
            return []
    return text

df["topic_vector"] = df["topic_vector"].apply(fix_vector)

df["topic_vector"].apply(lambda x: len(x)).value_counts()

df.to_csv("/content/drive/MyDrive/Skripsi4/dictionary/exploded_clean.csv", index=False)


## Directed M->D

In [16]:
def compute_od_m_to_d(proposals_df, experts_df):
    all_results = []

    for _, proposal in proposals_df.iterrows():
        proposal_vector = proposal["topic_vector"]
        mahasiswa = proposal["student_id"]
        id_proposal = proposal["proposal_id"]
        tahun_proposal = proposal["tahun"]

        for _, expert in expert_df.iterrows():
            expert_vector = expert["topic_vector"]
            dosen = expert["name"]
            id_dosen = expert["expert_id"]
            id_penelitian = expert["research_id"]
            tahun_penelitian = expert["pub_year"]
            weight = expert.get("author_weight")
            position= expert["author_position"]

            sim_mahasiswa = similarity_m_to_d(proposal_vector, expert_vector)
            score = sim_mahasiswa * weight

            all_results.append({
                "id_proposal": id_proposal,
                "id_penelitian": id_penelitian,
                "mahasiswa": mahasiswa,
                "dosen": dosen,
                "id_dosen": id_dosen,
                "posisi_author": position,
                "author_weight":weight,
                "tahun_proposal": tahun_proposal,
                "tahun_penelitian": tahun_penelitian,
                "OD(M→D)": round(score, 4)
            })
    df_scores = pd.DataFrame(all_results)
    df_scores = df_scores.sort_values(by=["id_proposal", "OD(M→D)"], ascending=[True, False])
    return df_scores


In [15]:
od_m2d_df = compute_od_m_to_d(proposal_df, expert_df)
print(od_m2d_df[["id_proposal", "mahasiswa","id_penelitian","dosen","id_dosen","posisi_author","author_weight", "OD(M→D)"]].head(30))

# od_m2d_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/od_m2d_df_topik.csv", index=False)

    id_proposal mahasiswa id_penelitian              dosen id_dosen  \
521          P1        S1          R333  Hasan Dwi Cahyono      D13   
520          P1        S1          R332  Hasan Dwi Cahyono      D13   
420          P1        S1          R253    Bambang Harjito       D1   
436          P1        S1          R266        Umi Salamah       D3   
741          P1        S1          R456    Bambang Harjito       D1   
489          P1        S1          R307            Wiharto      D12   
575          P1        S1          R363    Bambang Harjito       D1   
163          P1        S1           R97            Wiranto       D2   
83           P1        S1           R48    Bambang Harjito       D1   
137          P1        S1           R83       Esti Suryani       D8   
437          P1        S1          R267        Umi Salamah       D3   
270          P1        S1          R158      Heri Prasetyo      D15   
416          P1        S1          R251    Bambang Harjito       D1   
533   

## TOD M->D

In [17]:
# Hitung TOD(M→D)
def compute_tod_m_to_d(od_df, t=1, gamma=0.1, max_year_diff=5):
    od_df = od_df.copy()

    # Hitung selisih tahun
    od_df["selisih_tahun"] = od_df["tahun_proposal"] - od_df["tahun_penelitian"]

    # Filter: proposal harus lebih baru dari publikasi, dan selisih maksimal 5 tahun
    od_df = od_df[(od_df["selisih_tahun"] >= 0) & (od_df["selisih_tahun"] <= max_year_diff)].copy()

    # Hitung time decay factor
    od_df["time_decay_factor"] = od_df.apply(
        lambda row: time_decay(row["tahun_proposal"], row["tahun_penelitian"], t=t, gamma=gamma),
        axis=1
    )

    # Hitung TOD(M→D)
    od_df["TOD(M→D)"] = (od_df["OD(M→D)"] * od_df["time_decay_factor"]).round(4)

    return od_df



In [18]:
# Hitung TOD(M→D) Disortir
tod_m2d_df= compute_tod_m_to_d(od_m2d_df, t=1, gamma=0.1, max_year_diff=5)

tod_m2d_df = (
    tod_m2d_df
    .loc[tod_m2d_df.groupby(["id_proposal", "id_dosen"])["TOD(M→D)"].idxmax()]
    .reset_index(drop=True)
    .sort_values(by=["id_proposal", "TOD(M→D)"], ascending=[True, False])
)


# Lihat contoh hasil

print(tod_m2d_df[["id_proposal","mahasiswa", "id_penelitian","dosen","id_dosen", "OD(M→D)", "tahun_proposal","tahun_penelitian","selisih_tahun", "time_decay_factor", "TOD(M→D)"]].head(5))

# tod_m2d_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/tod_m2d_df_topik.csv", index=False)

   id_proposal mahasiswa id_penelitian            dosen id_dosen  OD(M→D)  \
3           P1        S1          R307          Wiharto      D12   0.2089   
9           P1        S1          R266      Umi Salamah       D3   0.2212   
8           P1        S1           R97          Wiranto       D2   0.1992   
0           P1        S1           R48  Bambang Harjito       D1   0.1990   
14          P1        S1           R53     Esti Suryani       D8   0.1382   

    tahun_proposal  tahun_penelitian  selisih_tahun  time_decay_factor  \
3             2021              2020              1                0.9   
9             2021              2018              3                0.7   
8             2021              2018              3                0.7   
0             2021              2018              3                0.7   
14            2021              2021              0                1.0   

    TOD(M→D)  
3     0.1880  
9     0.1548  
8     0.1394  
0     0.1393  
14    0.1382  


# OD D->M

In [19]:
def compute_od_d_to_m(proposals_df, experts_df):
    results = []

    for _, expert in experts_df.iterrows():
        expert_vector = expert["topic_vector"]
        dosen = expert["name"]
        id_dosen = expert["expert_id"]
        id_penelitian = expert["research_id"]
        weight = expert["author_weight"]
        position = expert["author_position"]
        tahun_penelitian = expert["pub_year"]

        for _, proposal in proposals_df.iterrows():
            proposal_vector = proposal["topic_vector"]
            mahasiswa = proposal["student_id"]
            id_proposal = proposal["proposal_id"]
            tahun_proposal = proposal["tahun"]

            # # Tambahkan filter tahun
            selisih_tahun = tahun_proposal - tahun_penelitian
            if 0 < selisih_tahun <= 5 and tahun_penelitian <= tahun_proposal:
               sim_dosen = similarity_d_to_m(expert_vector, proposal_vector)
               score = sim_dosen * weight
               results.append({
                    "id_proposal": id_proposal,
                    "id_penelitian": id_penelitian,
                    "mahasiswa": mahasiswa,
                    "dosen": dosen,
                    "id_dosen": id_dosen,
                    "posisi_author": position,
                    "author_weight": weight,
                    "tahun_proposal": tahun_proposal,
                    "tahun_penelitian": tahun_penelitian,
                    "OD(D→M)": round(score, 4),
                })

    df_scores = pd.DataFrame(results)
    df_scores = df_scores.loc[df_scores.groupby(["id_proposal","id_dosen"])["OD(D→M)"].idxmax()].reset_index(drop=True)
    df_scores = df_scores.sort_values(by=["id_dosen", "OD(D→M)"], ascending=[True, False])

    return df_scores

od_d2m_df = compute_od_d_to_m(proposal_df, expert_df)
print(od_d2m_df[["id_proposal", "mahasiswa","id_penelitian", "dosen", "id_dosen","posisi_author","author_weight","OD(D→M)"]].head(5))
# od_d2m_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/od_d2m_df_topik.csv", index=False)

     id_proposal mahasiswa id_penelitian            dosen id_dosen  \
100         P104      S104          R363  Bambang Harjito       D1   
392         P120      S120           R50  Bambang Harjito       D1   
1627         P64       S64          R363  Bambang Harjito       D1   
1579         P61       S61           R56  Bambang Harjito       D1   
1945         P82       S82          R344  Bambang Harjito       D1   

      posisi_author  author_weight  OD(D→M)  
100               1            1.0   0.9023  
392               1            1.0   0.8934  
1627              1            1.0   0.7232  
1579              1            1.0   0.5444  
1945              1            1.0   0.5355  


# Overlap

In [21]:
import pandas as pd

def combine_overlap_scores(df_m2d, df_d2m):
    # Ambil semua kolom yang dibutuhkan dari masing-masing arah
    top_m2d = df_m2d[["id_proposal", "mahasiswa", "id_dosen", "dosen", "TOD(M→D)"]]
    top_d2m = df_d2m[["id_proposal", "mahasiswa", "id_dosen", "dosen", "OD(D→M)"]]

    # Gabungkan kedua dataframe berdasarkan id_proposal dan id_dosen
    merged = pd.merge(top_m2d, top_d2m, on=["id_proposal", "id_dosen"], how="outer", suffixes=('_m2d', '_d2m'))

    # Gabungkan kolom nama mahasiswa dan dosen (dari salah satu sisi)
    merged["mahasiswa"] = merged["mahasiswa_m2d"].combine_first(merged["mahasiswa_d2m"])
    merged["dosen"] = merged["dosen_m2d"].combine_first(merged["dosen_d2m"])

    # Ganti NaN skor dengan 0
    merged["TOD(M→D)"] = merged["TOD(M→D)"].fillna(0)
    merged["OD(D→M)"] = merged["OD(D→M)"].fillna(0)

    # Tandai overlap jika skor dari dua arah ada (lebih dari 0)
    merged["overlap"] = (merged["TOD(M→D)"] > 0) & (merged["OD(D→M)"] > 0)

    # Hitung skor rata-rata hanya jika overlap, jika tidak maka 0
    merged["skor_rata2"] = merged.apply(
        lambda row: (row["TOD(M→D)"] + row["OD(D→M)"]) / 2 if row["overlap"] else 0, axis=1
    )

    # Urutkan berdasarkan skor rata-rata tertinggi untuk setiap proposal
    final_scores = merged.sort_values(by=["id_proposal", "skor_rata2"], ascending=[True, False])

    # Pilih kolom akhir yang relevan
    final_scores = final_scores[[
        "id_proposal", "mahasiswa", "dosen", "id_dosen",
        "TOD(M→D)", "OD(D→M)", "skor_rata2", "overlap"
    ]]

    return final_scores

# Contoh penggunaan:
df_final = combine_overlap_scores(tod_m2d_df, od_d2m_df)
print(df_final.head(30))
# df_final.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/overlap_directed_topik.csv", index=False)


   id_proposal mahasiswa               dosen id_dosen  TOD(M→D)  OD(D→M)  \
3           P1        S1             Wiharto      D12    0.1880   0.4377   
9           P1        S1         Umi Salamah       D3    0.1548   0.4486   
14          P1        S1        Esti Suryani       D8    0.1382   0.3555   
8           P1        S1             Wiranto       D2    0.1394   0.3501   
0           P1        S1     Bambang Harjito       D1    0.1393   0.3053   
6           P1        S1       Heri Prasetyo      D15    0.1187   0.2766   
11          P1        S1          Abdul Aziz       D5    0.0611   0.2370   
7           P1        S1     Ardhi Wijayanto      D16    0.0902   0.1573   
13          P1        S1      Wisnu Widiarto       D7    0.0602   0.1489   
12          P1        S1       Ristu Saptono       D6    0.0459   0.1313   
15          P1        S1    Sari Widya Sihwi       D9    0.0326   0.1380   
5           P1        S1     Haryono Setiadi      D14    0.0486   0.1075   
1           

# Ranking

In [22]:
def combine_overlap_scores_with_ranking(df_m2d, df_d2m):
    # Ambil semua skor dari kedua arah
    m2d = df_m2d[["id_proposal", "id_dosen","mahasiswa", "dosen", "TOD(M→D)"]]
    d2m = df_d2m[["id_proposal","id_dosen","mahasiswa","dosen", "OD(D→M)"]]

    # Outer join agar semua kombinasi muncul
    merged = pd.merge(m2d, d2m, on=["id_proposal", "id_dosen"], how="outer")

    # Tambahkan kolom nama dosen jika hilang (dari M→D arah saja)
    if "dosen_x" in merged.columns:
        merged["dosen"] = merged["dosen_x"].combine_first(merged.get("dosen_y"))
    elif "dosen" not in merged.columns:
        merged["dosen"] = None

    if "mahasiswa_x" in merged.columns:
        merged["mahasiswa"] = merged["mahasiswa_x"].combine_first(merged.get("mahasiswa_y"))
    elif "mahasiswa" not in merged.columns:
        merged["mahasiswa"] = None

    # Isi nilai NaN dengan 0 untuk penggabungan skor
    merged["TOD(M→D)"] = merged["TOD(M→D)"].fillna(0)
    merged["OD(D→M)"] = merged["OD(D→M)"].fillna(0)

    # Overlap = muncul di kedua arah
    merged["overlap"] = (merged["TOD(M→D)"] > 0) & (merged["OD(D→M)"] > 0)

    # Skor rata-rata jika overlap
    merged["skor_rata2"] = merged.apply(
        lambda row: (row["TOD(M→D)"] + row["OD(D→M)"]) / 2 if row["overlap"] else 0, axis=1
    )

       # Hitung ranking per proposal berdasarkan skor rata-rata (tanpa groupby + agg)
    merged["rank"] = merged.groupby("id_proposal")["skor_rata2"]\
                           .rank(ascending=False, method="dense")\
                           .astype(int)

    # Ambil kolom yang diinginkan dan urutkan
    result = merged.sort_values(by=["id_proposal", "rank"])[
        ["id_proposal","mahasiswa", "dosen", "id_dosen", "TOD(M→D)", "OD(D→M)", "skor_rata2", "overlap", "rank"]
    ]


    return result.sort_values(by=["id_proposal", "rank"])[
        ["id_proposal","mahasiswa", "dosen", "id_dosen", "TOD(M→D)", "OD(D→M)", "skor_rata2", "overlap", "rank"]
    ]


df_peringkat = combine_overlap_scores_with_ranking(tod_m2d_df, od_d2m_df)
# Filter hanya yang overlap == True
df_overlap_true = df_peringkat[df_peringkat["overlap"] == True]
print(df_overlap_true.head(20))

# df_overlap_true.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/rank_overlap_true_kata.csv", index=False)


   id_proposal mahasiswa               dosen id_dosen  TOD(M→D)  OD(D→M)  \
3           P1        S1             Wiharto      D12    0.1880   0.4377   
9           P1        S1         Umi Salamah       D3    0.1548   0.4486   
14          P1        S1        Esti Suryani       D8    0.1382   0.3555   
8           P1        S1             Wiranto       D2    0.1394   0.3501   
0           P1        S1     Bambang Harjito       D1    0.1393   0.3053   
6           P1        S1       Heri Prasetyo      D15    0.1187   0.2766   
11          P1        S1          Abdul Aziz       D5    0.0611   0.2370   
7           P1        S1     Ardhi Wijayanto      D16    0.0902   0.1573   
13          P1        S1      Wisnu Widiarto       D7    0.0602   0.1489   
12          P1        S1       Ristu Saptono       D6    0.0459   0.1313   
15          P1        S1    Sari Widya Sihwi       D9    0.0326   0.1380   
5           P1        S1     Haryono Setiadi      D14    0.0486   0.1075   
1           

In [23]:
from collections import defaultdict

# 1. Inisialisasi count untuk rank 1
rank1_count_directed = defaultdict(int)
final_assignment_directed = []

# 2. Tetapkan rank 1 dengan batas 15 kali per dosen
for pid in df_overlap_true["id_proposal"].unique():
    candidates = df_overlap_true[df_overlap_true["id_proposal"] == pid]
    candidates = candidates.sort_values(by="skor_rata2", ascending=False)

    assigned_rank1 = False
    for _, row in candidates.iterrows():
        dosen_id = row["id_dosen"]
        if rank1_count_directed[dosen_id] < 15:
            rank1_count_directed[dosen_id] += 1
            row_data = row.to_dict()
            row_data["rank"] = 1
            row_data["beban"] = rank1_count_directed[dosen_id]
            final_assignment_directed.append(row_data)
            assigned_rank1 = True
            break

    if not assigned_rank1:
        row = candidates.iloc[0].to_dict()
        dosen_id = row["id_dosen"]
        rank1_count_directed[dosen_id] += 1
        row["rank"] = 1
        row["beban"] = rank1_count_directed[dosen_id]
        final_assignment_directed.append(row)

# 3. Buat dataframe dari rank 1
rank1_directed = pd.DataFrame(final_assignment_directed)

# 4. Tambahkan rank 2–17 berdasarkan similarity, excl. dosen yang sudah dipakai di rank 1 untuk proposal yang sama
other_ranks = []

for pid in df_overlap_true["id_proposal"].unique():
    # Dapatkan dosen yang sudah dipakai sebagai rank 1
    used_dosen = rank1_directed[rank1_directed["id_proposal"] == pid]["id_dosen"].tolist()

    # Ambil kandidat lain untuk proposal ini
    candidates = df_overlap_true[(df_overlap_true["id_proposal"] == pid) & (~df_overlap_true["id_dosen"].isin(used_dosen))]
    candidates = candidates.sort_values(by="skor_rata2", ascending=False).reset_index(drop=True)

    for idx, (_, row) in enumerate(candidates.iterrows(), start=2):
        if idx > 17:
            break
        row_data = row.to_dict()
        row_data["rank"] = idx
        row_data["beban"] = rank1_count_directed[row_data["id_dosen"]]  # Beban hanya dihitung dari rank 1
        other_ranks.append(row_data)

# 5. Gabungkan rank1 dan other ranks
df_ranked_filtered = pd.concat([rank1_directed, pd.DataFrame(other_ranks)], ignore_index=True)
df_ranked_filtered = df_ranked_filtered.sort_values(by=["id_proposal", "rank"])
df_ranked_filtered = df_ranked_filtered[df_ranked_filtered["rank"] <= 17]

# Print the DataFrame
print(df_ranked_filtered)
# df_ranked_filtered.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/beban_directed_topik.csv", index=False)

     id_proposal mahasiswa            dosen id_dosen  TOD(M→D)  OD(D→M)  \
0             P1        S1          Wiharto      D12    0.1880   0.4377   
142           P1        S1      Umi Salamah       D3    0.1548   0.4486   
143           P1        S1     Esti Suryani       D8    0.1382   0.3555   
144           P1        S1          Wiranto       D2    0.1394   0.3501   
145           P1        S1  Bambang Harjito       D1    0.1393   0.3053   
...          ...       ...              ...      ...       ...      ...   
2243         P99       S99      Umi Salamah       D3    0.0889   0.1747   
2244         P99       S99    Heri Prasetyo      D15    0.0633   0.1598   
2245         P99       S99  Haryono Setiadi      D14    0.0419   0.0350   
2246         P99       S99   Afrizal Doewes      D10    0.0229   0.0463   
2247         P99       S99   Wisnu Widiarto       D7    0.0123   0.0361   

      skor_rata2  overlap  rank  beban  
0        0.31285     True     1      1  
142      0.30170 

# Evaluasi

In [24]:
from sklearn.metrics import precision_score, recall_score

# Load the true labels DataFrame
true_label_df = pd.read_csv("/content/drive/MyDrive/Skripsi3/Dataset/true_labels.csv")

# Ubah kolom author, author2, author3 menjadi lowercase
true_label_df["author1"] = true_label_df["examiner_1"].astype(str).str.strip()
true_label_df["author2"] = true_label_df["examiner_2"].astype(str).str.strip()
true_label_df["author3"] = true_label_df["examiner_3"].astype(str).str.strip()



coba

## Evaluasi Baru

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def evaluate_ordered_recommendation_directed(rank_all_df, true_label_df, top_ns=[3, 5, 7, 10]):
    summary = []

    for TOP_N in top_ns:
        # Filter Top-N dan urutkan
        top_n_df = rank_all_df[rank_all_df["rank"] <= TOP_N]
        rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "skor_rata2"], ascending=[True, True, False])
        rec_df = rec_df.drop_duplicates(subset=["id_proposal", "rank","skor_rata2"])
        rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
        rec_pivot.columns.name = None
        rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

        # Gabungkan dengan ground truth
        merged_df = pd.merge(
            rec_pivot,
            true_label_df.rename(columns={"proposal_id": "id_proposal"}),
            on="id_proposal",
            how="left"
        )

        # Recall keberadaan (tidak memperhatikan urutan)
        def recall_of_existence(row):
            true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
            pred_set = {row.get(f"rec_{i}") for i in range(1, TOP_N + 1) if row.get(f"rec_{i}")}
            return len(true_set.intersection(pred_set)) / 3

        merged_df[f'recall_of_existence@{TOP_N}'] = merged_df.apply(recall_of_existence, axis=1)

        # Recall per posisi dengan urutan diperhatikan (rec_i harus sama dengan author_i)
        recall_pos = {1: [], 2: [], 3: []}
        for _, row in merged_df.iterrows():
            for pos in [1, 2, 3]:
                examiner = row.get(f'author{pos}')
                rec = row.get(f'rec_{pos}') if pos <= TOP_N else None
                hit = int(pd.notna(examiner) and pd.notna(rec) and examiner == rec)
                recall_pos[pos].append(hit)

        # Tambahkan recall ke DataFrame
        for pos in [1, 2, 3]:
            merged_df[f'recall_pos{pos}_ordered@{TOP_N}'] = recall_pos[pos]

        recall_pos_mean = {pos: np.mean(recall_pos[pos]) for pos in [1, 2, 3]}

        # Euclidean distance antar posisi (penalti posisi meleset)
        distances = []
        for _, row in merged_df.iterrows():
            true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
            pred_authors = [row.get(f'rec_{i}', None) for i in range(1, TOP_N + 1)]
            distance = 0
            max_penalty = TOP_N
            for i, true_author in enumerate(true_authors):
                if pd.isna(true_author) or true_author == '':
                    continue
                try:
                    pred_pos = pred_authors.index(true_author)
                    pos_diff = pred_pos - i
                    distance += pos_diff ** 2
                except ValueError:
                    distance += max_penalty ** 2
            distances.append(np.sqrt(distance))

        scaler = MinMaxScaler()
        norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
        merged_df[f'norm_euclidean@{TOP_N}'] = norm_dists

        # Ringkasan metrik
        summary.append({
            'Top-N': TOP_N,
            'Mean_Recall_Existence': merged_df[f'recall_of_existence@{TOP_N}'].mean(),
            'Recall_Pos_1_Ordered': recall_pos_mean[1],
            'Recall_Pos_2_Ordered': recall_pos_mean[2],
            'Recall_Pos_3_Ordered': recall_pos_mean[3],
            'Avg_Normalized_Euclidean': np.mean(norm_dists)
        })

    return pd.DataFrame(summary)

result_df_directed = evaluate_ordered_recommendation_directed(df_overlap_true, true_label_df)
print(result_df_directed)



   Top-N  Mean_Recall_Existence  Recall_Pos_1_Ordered  Recall_Pos_2_Ordered  \
0      3               0.340376              0.380282              0.056338   
1      5               0.511737              0.380282              0.056338   
2      7               0.636150              0.380282              0.056338   
3     10               0.781690              0.380282              0.056338   

   Recall_Pos_3_Ordered  Avg_Normalized_Euclidean  
0              0.091549                  0.822388  
1              0.091549                  0.710031  
2              0.091549                  0.625719  
3              0.091549                  0.555407  


## Evaluasi per proposal Baru

In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def evaluate_per_proposal_directed(rank_all_df, true_label_df, top_n=3):
    # Filter dan urutkan
    top_n_df = rank_all_df[rank_all_df["rank"] <= top_n]
    rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "skor_rata2"], ascending=[True, True, False])
    rec_df = rec_df.drop_duplicates(subset=["id_proposal", "rank","skor_rata2"])
    rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
    rec_pivot.columns.name = None
    rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

    # Gabung dengan label kebenaran
    merged_df = pd.merge(
        rec_pivot,
        true_label_df.rename(columns={"proposal_id": "id_proposal"}),
        on="id_proposal",
        how="left"
    )

    # Recall of existence (abaikan urutan)
    def recall_of_existence(row):
        true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
        pred_set = {row.get(f"rec_{i}") for i in range(1, top_n + 1) if row.get(f"rec_{i}")}
        return len(true_set.intersection(pred_set)) / 3

    merged_df[f'recall_of_existence@{top_n}'] = merged_df.apply(recall_of_existence, axis=1)

    # Recall berdasarkan posisi (urutan harus sama)
    for pos in [1, 2, 3]:
        merged_df[f'recall_pos{pos}_ordered@{top_n}'] = merged_df.apply(
            lambda row: int(
                pd.notna(row.get(f'author{pos}')) and
                pd.notna(row.get(f'rec_{pos}')) and
                row.get(f'author{pos}') == row.get(f'rec_{pos}')
            ) if pos <= top_n else 0,
            axis=1
        )

    # Euclidean distance penalti posisi
    distances = []
    for _, row in merged_df.iterrows():
        true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
        pred_authors = [row.get(f'rec_{i}', None) for i in range(1, top_n + 1)]
        distance = 0
        max_penalty = top_n
        for i, true_author in enumerate(true_authors):
            if pd.isna(true_author) or true_author == '':
                continue
            try:
                pred_pos = pred_authors.index(true_author)
                pos_diff = pred_pos - i
                distance += pos_diff ** 2
            except ValueError:
                distance += max_penalty ** 2
        distances.append(np.sqrt(distance))

    # Normalisasi jarak
    scaler = MinMaxScaler()
    norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
    merged_df[f'norm_euclidean@{top_n}'] = norm_dists

    # Ambil kolom evaluasi
    result_df = merged_df[["id_proposal",
                           f'recall_of_existence@{top_n}',
                           f'recall_pos1_ordered@{top_n}',
                           f'recall_pos2_ordered@{top_n}',
                           f'recall_pos3_ordered@{top_n}',
                           f'norm_euclidean@{top_n}']].copy()

    return result_df


eval_per_proposal_directed_3 = evaluate_per_proposal_directed(df_overlap_true, true_label_df, top_n=3)
eval_per_proposal_directed_5 = evaluate_per_proposal_directed(df_overlap_true, true_label_df, top_n=5)
eval_per_proposal_directed_7 = evaluate_per_proposal_directed(df_overlap_true, true_label_df, top_n=7)
eval_per_proposal_directed_10 = evaluate_per_proposal_directed(df_overlap_true, true_label_df, top_n=10)

# eval_per_proposal_directed_3.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/hasil_eval_3_directed.csv", index=False)
# eval_per_proposal_directed_5.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/hasil_eval_5_directed.csv", index=False)
# eval_per_proposal_directed_7.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/hasil_eval_7_directed.csv", index=False)
# eval_per_proposal_directed_10.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/hasil_eval_10_directed.csv", index=False)

# Cosine

In [27]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Konversi kolom topic_vector menjadi 2D array
expert_vectors = np.vstack(expert_df["topic_vector"].values)
proposal_vectors = np.vstack(proposal_df["topic_vector"].values)

# Hitung similarity
similarity_matrix = cosine_similarity(expert_vectors, proposal_vectors)

print(similarity_matrix)


[[0.02116158 0.19097795 0.1399585  ... 0.02030642 0.02484113 0.05370035]
 [0.02116158 0.19097795 0.1399585  ... 0.02030642 0.02484113 0.05370035]
 [0.01516028 0.02125837 0.02516963 ... 0.06561315 0.05264645 0.05729351]
 ...
 [0.14608201 0.02545677 0.09075104 ... 0.03309006 0.03286479 0.05853668]
 [0.20352237 0.02324828 0.07729209 ... 0.03353393 0.0428044  0.05218332]
 [0.11509452 0.01096047 0.0180661  ... 0.02063077 0.02465363 0.02465597]]


In [30]:
similarity_cosine_df = []

for i, (_, mahasiswa) in enumerate(proposal_df.iterrows()):
    for j, (_, dosen) in enumerate(expert_df.iterrows()):
        similarity_cosine_df.append({
            "id_proposal": mahasiswa["proposal_id"],
            "mahasiswa": mahasiswa["student_id"],
            "id_dosen": dosen["expert_id"],
            "id_penelitian": dosen["research_id"],
            "tahun_proposal": mahasiswa["tahun"],
            "tahun_penelitian": dosen["pub_year"],
            "selisih_tahun": mahasiswa["tahun"] - dosen["pub_year"],
            "dosen": dosen["name"],
           "author_position": dosen["author_position"],
            "similarity_score" : similarity_matrix[j, i]


        })

similarity_cosine_df = pd.DataFrame(similarity_cosine_df)

# Filter sesuai kondisi
similarity_cosine_df = similarity_cosine_df[
    (similarity_cosine_df["tahun_proposal"] > similarity_cosine_df["tahun_penelitian"]) &
    (similarity_cosine_df["selisih_tahun"] <= 5)
].copy()


# Tampilkan hasil
print(similarity_cosine_df[["id_proposal","mahasiswa","id_penelitian", "id_dosen","selisih_tahun", "dosen","author_position", "similarity_score" ,]])
# similarity_cosine_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/14/hasil_cosine_topik.csv", index=False)


       id_proposal mahasiswa id_penelitian id_dosen  selisih_tahun  \
3               P1        S1            R3       D5              5   
4               P1        S1            R3       D1              5   
5               P1        S1            R4       D5              4   
6               P1        S1            R4       D8              4   
7               P1        S1            R5       D5              3   
...            ...       ...           ...      ...            ...   
107846        P142      S142          R425       D2              2   
107847        P142      S142          R426       D6              2   
107849        P142      S142          R428      D14              2   
107850        P142      S142          R429      D14              2   
107851        P142      S142          R429       D6              2   

                  dosen  author_position  similarity_score  
3            Abdul Aziz                1          0.017203  
4       Bambang Harjito              

# Pemeringkatan

In [38]:
# Ambil baris dengan similarity tertinggi untuk kombinasi unik id_proposal dan id_dosen
idx = similarity_cosine_df.groupby(["id_proposal", "id_dosen"])["similarity_score"].idxmax()
similarity_cosine = similarity_cosine_df.loc[idx]

# Ranking ulang berdasarkan proposal
similarity_cosine_df["rank"] = similarity_cosine_df.groupby("id_proposal")["similarity_score"] \
                                     .rank(method="first", ascending=False).astype(int)

# Urutkan
similarity_cosine_df = similarity_cosine_df.sort_values(["id_proposal", "rank"])
similarity_cosine_df = similarity_cosine_df[similarity_cosine_df["rank"] <= 17]


# Tampilkan
print(similarity_cosine_df[["id_proposal", "mahasiswa","tahun_proposal","id_penelitian", "tahun_penelitian", "id_dosen", "dosen","selisih_tahun", "author_position","similarity_score","rank"]])
# similarity_cosine_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/14/cosine/similarity_cosine_kata_baru_setelah filter.csv", index=False)


      id_proposal mahasiswa  tahun_proposal id_penelitian  tahun_penelitian  \
137            P1        S1            2021           R83              2016   
138            P1        S1            2021           R83              2016   
163            P1        S1            2021           R97              2018   
164            P1        S1            2021           R97              2018   
84             P1        S1            2021           R48              2018   
...           ...       ...             ...           ...               ...   
74550         P99       S99            2024           R42              2023   
74815         P99       S99            2024          R202              2023   
74816         P99       S99            2024          R202              2023   
74685         P99       S99            2024          R116              2021   
74686         P99       S99            2024          R116              2021   

      id_dosen              dosen  selisih_tahun  a

In [39]:
from collections import defaultdict

# 1. Inisialisasi count untuk rank 1
rank1_count = defaultdict(int)
final_assignments = []

# 2. Tetapkan rank 1 dengan batas 15 kali per dosen
for pid in similarity_cosine_df["id_proposal"].unique():
    candidates = similarity_cosine_df[similarity_cosine_df["id_proposal"] == pid]
    candidates = candidates.sort_values(by="similarity_score", ascending=False)

    assigned_rank1 = False
    for _, row in candidates.iterrows():
        dosen_id = row["id_dosen"]
        if rank1_count[dosen_id] < 15:
            rank1_count[dosen_id] += 1
            row_data = row.to_dict()
            row_data["rank"] = 1
            row_data["beban"] = rank1_count[dosen_id]
            final_assignments.append(row_data)
            assigned_rank1 = True
            break

    if not assigned_rank1:
        row = candidates.iloc[0].to_dict()
        dosen_id = row["id_dosen"]
        rank1_count[dosen_id] += 1
        row["rank"] = 1
        row["beban"] = rank1_count[dosen_id]
        final_assignments.append(row)

# 3. Buat dataframe dari rank 1
rank1_df = pd.DataFrame(final_assignments)

# 4. Tambahkan rank 2–17 berdasarkan similarity, excl. dosen yang sudah dipakai di rank 1 untuk proposal yang sama
other_ranks = []

for pid in similarity_cosine_df["id_proposal"].unique():
    # Dapatkan dosen yang sudah dipakai sebagai rank 1
    used_dosen = rank1_df[rank1_df["id_proposal"] == pid]["id_dosen"].tolist()

    # Ambil kandidat lain untuk proposal ini
    candidates = similarity_cosine_df[(similarity_cosine_df["id_proposal"] == pid) & (~similarity_cosine_df["id_dosen"].isin(used_dosen))]
    candidates = candidates.sort_values(by="similarity_score", ascending=False).reset_index(drop=True)

    for idx, (_, row) in enumerate(candidates.iterrows(), start=2):
        if idx > 17:
            break
        row_data = row.to_dict()
        row_data["rank"] = idx
        row_data["beban"] = rank1_count[row_data["id_dosen"]]  # Beban hanya dihitung dari rank 1
        other_ranks.append(row_data)

# 5. Gabungkan rank1 dan other ranks
rank_all_df = pd.concat([rank1_df, pd.DataFrame(other_ranks)], ignore_index=True)
rank_all_df = rank_all_df.sort_values(by=["id_proposal", "rank"])

# 6. Tampilkan hasil
print(rank_all_df[["id_proposal", "mahasiswa", "id_dosen", "dosen", "author_position","similarity_score", "rank", "beban"]])
rank_all_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/beban_cosine_topik.csv", index=False)

     id_proposal mahasiswa id_dosen              dosen  author_position  \
0             P1        S1       D8       Esti Suryani                1   
142           P1        S1       D5         Abdul Aziz                2   
143           P1        S1       D2            Wiranto                1   
144           P1        S1       D1    Bambang Harjito                1   
145           P1        S1       D3        Umi Salamah                1   
...          ...       ...      ...                ...              ...   
2009         P99       S99      D11            Winarno                2   
2010         P99       S99      D11            Winarno                1   
2011         P99       S99      D13  Hasan Dwi Cahyono                2   
2012         P99       S99      D12            Wiharto                1   
2013         P99       S99       D8       Esti Suryani                2   

      similarity_score  rank  beban  
0             0.765764     1      1  
142           0.765764 

In [43]:
# Urutkan berdasarkan id_proposal, similarity_score_akhir (descending), dan author_position (ascending)
similarity_cosine_df_baru = rank_all_df.sort_values(
    by=["id_proposal", "similarity_score", "author_position"],
    ascending=[True, False, True]
)

# Tambahkan kolom rank untuk setiap id_proposal
similarity_cosine_df_baru["rank"] = similarity_cosine_df_baru.groupby("id_proposal").cumcount() + 1

print(similarity_cosine_df_baru[["id_proposal", "mahasiswa","tahun_proposal","id_penelitian", "tahun_penelitian", "id_dosen", "dosen","selisih_tahun", "author_position","similarity_score","rank","beban"]])
similarity_cosine_df_baru.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/beban_cosine_topik_urut_author.csv", index=False)

     id_proposal mahasiswa  tahun_proposal id_penelitian  tahun_penelitian  \
0             P1        S1            2021           R83              2016   
142           P1        S1            2021           R83              2016   
143           P1        S1            2021           R97              2018   
144           P1        S1            2021           R48              2018   
145           P1        S1            2021          R266              2018   
...          ...       ...             ...           ...               ...   
2009         P99       S99            2024           R42              2023   
2010         P99       S99            2024          R202              2023   
2011         P99       S99            2024          R202              2023   
2012         P99       S99            2024          R116              2021   
2013         P99       S99            2024          R116              2021   

     id_dosen              dosen  selisih_tahun  author_positio

# Evaluasi Cosine

In [41]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


def evaluate_ordered_recommendation_cosine(rank_all_df, true_label_df, top_ns=[3, 5, 7, 10]):
    summary = []

    for TOP_N in top_ns:
        # Filter Top-N dan urutkan
        top_n_df = rank_all_df[rank_all_df["rank"] <= TOP_N]
        rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "similarity_score"], ascending=[True, True, False])
        rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
        rec_pivot.columns.name = None
        rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

        # Gabungkan dengan ground truth
        merged_df = pd.merge(
            rec_pivot,
            true_label_df.rename(columns={"proposal_id": "id_proposal"}),
            on="id_proposal",
            how="left"
        )

        # Recall keberadaan (tidak memperhatikan urutan)
        def recall_of_existence(row):
            true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
            pred_set = {row.get(f"rec_{i}") for i in range(1, TOP_N + 1) if row.get(f"rec_{i}")}
            return len(true_set.intersection(pred_set)) / 3

        merged_df[f'recall_of_existence@{TOP_N}'] = merged_df.apply(recall_of_existence, axis=1)

        # Recall per posisi dengan urutan diperhatikan (rec_i harus sama dengan author_i)
        recall_pos = {1: [], 2: [], 3: []}
        for _, row in merged_df.iterrows():
            for pos in [1, 2, 3]:
                examiner = row.get(f'author{pos}')
                rec = row.get(f'rec_{pos}') if pos <= TOP_N else None
                hit = int(pd.notna(examiner) and pd.notna(rec) and examiner == rec)
                recall_pos[pos].append(hit)

        # Tambahkan recall ke DataFrame
        for pos in [1, 2, 3]:
            merged_df[f'recall_pos{pos}_ordered@{TOP_N}'] = recall_pos[pos]

        recall_pos_mean = {pos: np.mean(recall_pos[pos]) for pos in [1, 2, 3]}

        # Euclidean distance antar posisi (penalti posisi meleset)
        distances = []
        for _, row in merged_df.iterrows():
            true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
            pred_authors = [row.get(f'rec_{i}', None) for i in range(1, TOP_N + 1)]
            distance = 0
            max_penalty = TOP_N
            for i, true_author in enumerate(true_authors):
                if pd.isna(true_author) or true_author == '':
                    continue
                try:
                    pred_pos = pred_authors.index(true_author)
                    pos_diff = pred_pos - i
                    distance += pos_diff ** 2
                except ValueError:
                    distance += max_penalty ** 2
            distances.append(np.sqrt(distance))

        scaler = MinMaxScaler()
        norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
        merged_df[f'norm_euclidean@{TOP_N}'] = norm_dists

        # Ringkasan metrik
        summary.append({
            'Top-N': TOP_N,
            'Mean_Recall_Existence': merged_df[f'recall_of_existence@{TOP_N}'].mean(),
            'Recall_Pos_1_Ordered': recall_pos_mean[1],
            'Recall_Pos_2_Ordered': recall_pos_mean[2],
            'Recall_Pos_3_Ordered': recall_pos_mean[3],
            'Avg_Normalized_Euclidean': np.mean(norm_dists)
        })

    return pd.DataFrame(summary)

result_df = evaluate_ordered_recommendation_cosine(similarity_cosine_df, true_label_df)
print(result_df)



   Top-N  Mean_Recall_Existence  Recall_Pos_1_Ordered  Recall_Pos_2_Ordered  \
0      3               0.244131              0.309859              0.021127   
1      5               0.380282              0.309859              0.021127   
2      7               0.488263              0.309859              0.021127   
3     10               0.551643              0.309859              0.021127   

   Recall_Pos_3_Ordered  Avg_Normalized_Euclidean  
0              0.070423                  0.710102  
1              0.070423                  0.686285  
2              0.070423                  0.647736  
3              0.070423                  0.616253  


## per proposal baru

In [42]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def evaluate_per_proposal_cosine(rank_all_df, true_label_df, top_n=3):
    # Filter dan urutkan
    top_n_df = rank_all_df[rank_all_df["rank"] <= top_n]
    rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "similarity_score"], ascending=[True, True, False])
    rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
    rec_pivot.columns.name = None
    rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

    # Gabung dengan label kebenaran
    merged_df = pd.merge(
        rec_pivot,
        true_label_df.rename(columns={"proposal_id": "id_proposal"}),
        on="id_proposal",
        how="left"
    )

    # Recall of existence (abaikan urutan)
    def recall_of_existence(row):
        true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
        pred_set = {row.get(f"rec_{i}") for i in range(1, top_n + 1) if row.get(f"rec_{i}")}
        return len(true_set.intersection(pred_set)) / 3

    merged_df[f'recall_of_existence@{top_n}'] = merged_df.apply(recall_of_existence, axis=1)

    # Recall berdasarkan posisi (urutan harus sama)
    for pos in [1, 2, 3]:
        merged_df[f'recall_pos{pos}_ordered@{top_n}'] = merged_df.apply(
            lambda row: int(
                pd.notna(row.get(f'author{pos}')) and
                pd.notna(row.get(f'rec_{pos}')) and
                row.get(f'author{pos}') == row.get(f'rec_{pos}')
            ) if pos <= top_n else 0,
            axis=1
        )

    # Euclidean distance penalti posisi
    distances = []
    for _, row in merged_df.iterrows():
        true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
        pred_authors = [row.get(f'rec_{i}', None) for i in range(1, top_n + 1)]
        distance = 0
        max_penalty = top_n
        for i, true_author in enumerate(true_authors):
            if pd.isna(true_author) or true_author == '':
                continue
            try:
                pred_pos = pred_authors.index(true_author)
                pos_diff = pred_pos - i
                distance += pos_diff ** 2
            except ValueError:
                distance += max_penalty ** 2
        distances.append(np.sqrt(distance))

    # Normalisasi jarak
    scaler = MinMaxScaler()
    norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
    merged_df[f'norm_euclidean@{top_n}'] = norm_dists

    # Ambil kolom evaluasi
    result_df = merged_df[["id_proposal",
                           f'recall_of_existence@{top_n}',
                           f'recall_pos1_ordered@{top_n}',
                           f'recall_pos2_ordered@{top_n}',
                           f'recall_pos3_ordered@{top_n}',
                           f'norm_euclidean@{top_n}']].copy()

    return result_df


eval_per_proposal_cosine_3 = evaluate_per_proposal_cosine(similarity_cosine_df, true_label_df, top_n=3)
eval_per_proposal_cosine_5 = evaluate_per_proposal_cosine(similarity_cosine_df, true_label_df, top_n=5)
eval_per_proposal_cosine_7 = evaluate_per_proposal_cosine(similarity_cosine_df, true_label_df, top_n=7)
eval_per_proposal_cosine_10 = evaluate_per_proposal_cosine(similarity_cosine_df, true_label_df, top_n=10)

# eval_per_proposal_cosine_3.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/hasil_eval_3_cosine_topik.csv", index=False)
# eval_per_proposal_cosine_5.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/hasil_eval_5_cosine_topik.csv", index=False)
# eval_per_proposal_cosine_7.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/hasil_eval_7_cosine_topik.csv", index=False)
# eval_per_proposal_cosine_10.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/hasil_eval_10_cosine_topik.csv", index=False)